In [109]:
import pandas as pd
import glob

## Broad trends
Data across sites

In [150]:
df  = pd.read_excel('Total Numbers.xlsx', sheetname='Sheet1', header=None)

def gap():
    gapdf = df[[0, 1]].copy()
    gapdf = gapdf.iloc[0:4880]
    gapdf.columns = ['artlength', 'gender']
    gapdf['gender'] = gapdf['gender'].astype('category')
    return gapdf

def topic():
    topicdf = df[[24, 25]].copy()
    topicdf.columns = topicdf.iloc[0]
    topicdf = topicdf.reindex(topicdf.index.drop(0))
    topicdf['category'] = topicdf['category'].astype('category')
    topicdf['vals'] = 1
    return topicdf

def writers():
    writdf = df[[12, 17]].copy()
    writdf = writdf.dropna(how='all')
    writdf.columns = ['Men', 'Women']
    return writdf

In [156]:
print 'Number of men and women in the dataset:'
print writers().count()

print '\nArticles written by men and women:'
count = gap().groupby('gender').count()
print count

print '\nGap:'
print count.loc['m'].values[0] - count.loc['w'].values[0], 
print 'articles, or %s times as many stories for men.' % (float(count.loc['m'].values[0]) / float(count.loc['w'].values[0]))

print '\nMean article length:'
mean = gap().groupby('gender').mean()
print mean

print '\nTotal words:'
words = gap().groupby('gender').sum()
print words

print '\nGap:'
print words.loc['m'].values[0] - words.loc['w'].values[0],
print 'words, or %s times as many words for men.' % (float(words.loc['m'].values[0]) / float(words.loc['w'].values[0]))

Number of men and women in the dataset:
Men      301
Women    149
dtype: int64

Articles written by men and women:
        artlength
gender           
m            3493
w            1387

Gap:
2106 articles, or 2.5183850036 times as many stories for men.

Mean article length:
         artlength
gender            
m       481.921844
w       544.108147

Total words:
        artlength
gender           
m       1683353.0
w        754678.0

Gap:
928675.0 words, or 2.23055793332 times as many words for men.


In [122]:
grouped = topic().groupby(['category', 'byline']).count()

index = list(grouped.index)
index = [i[0] for i in index]
index = list(set(index))

diffs = [(i, 
          grouped.loc[(i, 'w'), 'vals'] / (grouped.loc[(i, 'm'), 'vals'] + grouped.loc[(i, 'w'), 'vals']), 
          grouped.loc[(i, 'm'), 'vals'] + grouped.loc[(i, 'w'), 'vals']) for i in index]

topics = pd.DataFrame(diffs, columns=['Topic', '% Women', 'n']).sort_values(by='% Women', ascending=False)
topics = topics.dropna()
topics = topics.drop(topics.index[[6, 2]]).reset_index(drop=True)
topics

,Topic,% Women,n
0,Design,0.687500,80.0
1,Space,0.540230,261.0
2,Health,0.401487,269.0
3,Environment,0.369565,138.0
4,n/t,0.345566,327.0
5,Business,0.316364,275.0
6,Film,0.292035,113.0
7,Science,0.290909,55.0
8,Apps,0.286275,255.0
9,Government,0.282238,411.0


## Site-specific data

In [136]:
filenames = glob.glob("*.csv")
engadget, motherboard, techcrunch, verge, wired = (pd.read_csv(name, usecols=['byline', 'body']).groupby('byline').count() for name in filenames)
sites = [engadget, motherboard, techcrunch, verge, wired]
sitenames = {engadget.iloc[0].values[0] : 'Engadget',
            motherboard.iloc[0].values[0] : 'Motherboard',
            techcrunch.iloc[0].values[0] : 'Techcrunch',
            verge.iloc[0].values[0] : 'Verge',
            wired.iloc[0].values[0] : 'Wired'}
for site in sites:
    print '\n%s gap:' % sitenames[site.iloc[0].values[0]]
    print site.loc['m'].values[0] - site.loc['w'].values[0], 
    print 'articles, or %s times as many stories for men.' % (float(site.loc['m'].values[0]) / float(site.loc['w'].values[0]))



Engadget gap:
692 articles, or 3.19682539683 times as many stories for men.

Motherboard gap:
104 articles, or 1.97196261682 times as many stories for men.

Techcrunch gap:
402 articles, or 2.75545851528 times as many stories for men.

Verge gap:
678 articles, or 2.77952755906 times as many stories for men.

Wired gap:
172 articles, or 1.6231884058 times as many stories for men.


In [160]:
filenames = glob.glob("*.csv")
engadget, motherboard, techcrunch, verge, wired = (pd.read_csv(name, usecols=['byline', 'body']).groupby('byline').mean() for name in filenames)
sites = [engadget, motherboard, techcrunch, verge, wired]
sitenames = {engadget.iloc[0].values[0] : 'Engadget',
            motherboard.iloc[0].values[0] : 'Motherboard',
            techcrunch.iloc[0].values[0] : 'Techcrunch',
            verge.iloc[0].values[0] : 'Verge',
            wired.iloc[0].values[0] : 'Wired'}
for site in sites:
    print '\n%s mean article lengths:' % sitenames[site.iloc[0].values[0]]
    print site


Engadget mean article lengths:
              body
byline            
m       343.359484
w       375.060317

Motherboard mean article lengths:
              body
byline            
m       711.601896
w       622.485981

Techcrunch mean article lengths:
              body
byline            
m       441.122029
w       534.956332

Verge mean article lengths:
              body
byline            
m       418.330500
w       475.188976

Wired mean article lengths:
              body
byline            
m       871.687500
w       782.271739
